In [ ]:
import pandas as pd
import time
import pickle

### Get full target

In [ ]:
all_tweets = pd.read_csv('congress_tweets.csv'); len(all_tweets)

In [ ]:
bots_df = pd.read_csv('bots_df.csv')
bots_df = bots_df[bots_df.is_bot == 1]

In [ ]:
all_tweets = all_tweets[~all_tweets.screen_name.isin(bots_df.screen_name.values)]

In [ ]:
single_targets = all_tweets[(all_tweets.directed_at_screen_name.notna()) | (all_tweets.mentions == 'set()')]

In [ ]:
multiple_targets = all_tweets[all_tweets.directed_at_screen_name.isna()]; len(multiple_targets)

In [ ]:
meta_dict_congress = pickle.load(open("meta_dict_congress.pkl", "rb"))


In [ ]:
rows = []

In [ ]:
%%time
for idx,row in multiple_targets.iterrows():
    if idx % 50000 == 0 and idx > 0:
        print(idx)
        time.sleep(30)
    for name in eval(row.mentions_congress_list):
        rows = row.copy()
        rows.directed_at_screen_name = name
        rows.directed_at_congress = meta_dict[name]['type']          
        rows.directed_at_gender = meta_dict[name]['gender']
        rows.directed_at_state = meta_dict[name]['state']
        rows.directed_at_interbloque = meta_dict[name]['interbloque']
        rows.append(row2)

In [ ]:
single_targets = single_targets.append(rows).reset_index(drop=True)

In [ ]:
print(len(all_tweets), len(single_targets))

### Merge with unique tweet index

In [ ]:
id_to_idx = pd.read_csv('../2.perspective/id_to_idx.csv')

In [ ]:
single_targets2 = single_targets2.merge(id_to_idx, how='left', on='id')

In [ ]:
single_targets2.to_csv('processed/congress_tweets_full_targets.csv', index=False)

### Get target precision

In [ ]:
all_tweets = pd.read_csv('processed/congress_tweets_full_targets.csv')

In [ ]:
meta = pd.read_csv('metadata.csv')
# Replies to politicians are address to them

all_tweets.loc[all_tweets.in_reply_to_screen_name.isin(meta.screen_name),"target_precision"] = "High"

In [ ]:
# Number of mentions
all_tweets.mentions = all_tweets.mentions.fillna('set()')
all_tweets['n_mentions'] = all_tweets.mentions.apply(lambda x: len(eval(x)))

In [ ]:
all_tweets.loc[all_tweets.n_mentions==1,"target_precision"] = "High"

In [ ]:
# Number of mentions politicians
all_tweets.mentions_politicians = all_tweets.mentions_politicians.fillna('set()')
all_tweets['n_mentions_poilticians'] = all_tweets.mentions_politicians.apply(lambda x: len(eval(x)))

In [ ]:
all_tweets.loc[(all_tweets.target_precision != "High")&\
               (all_tweets.n_mentions_poilticians==1)&\
               (all_tweets.n_mentions > 1),"target_precision"] = "Medium"

In [ ]:
all_tweets.loc[all_tweets.target_precision.isna(),"target_precision"] = "Low"

In [ ]:
all_tweets.target_precision.value_counts()

In [ ]:
len(all_tweets)

In [ ]:
all_tweets.to_csv('processed/congress_tweets_full_targets.csv', index=False)

In [ ]:
all_tweets.drop_duplicates('id')[['id','target_precision']].to_csv('processed/id_to_target.csv',index=False)